In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "austin_texas_sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

SCHEMA = schema.schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']




expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]


mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Ave.": "Avenue"
            
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

            

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_postcode(postcode):
    """This function is used to clear wrong postcode format from
    the postcode containing anything other then 5 digits.
   """
    # regular expressions to capture only numbers in the postcode
    postcode_re = re.compile(r'\d{3,6}', re.IGNORECASE)  
    
    if not postcode_re.search(postcode):
    # if postcode contains other symbols than digits do the following:
    # Only digits will be shown in the postcode
        print 'BEFORE'
        print postcode

        # replace is a python string method, not a regex method:
    
        postcode=re.sub(r'\D', "", postcode)
        print 'AFTER'
        print postcode
    return postcode
  

#This function takes a string with street name as an argument and return the fixed name 
def update_name(name, mapping):
    for search_error in mapping:
        # In this case, the arguments are '(replace with, string to search)'       
        if search_error in name:
             print "BEFORE"
             print name
             name = re.sub(r'\b' + search_error + r'\b\.?', mapping[search_error], name)
             print "AFTER"
             print name
    return name

def correct_k(k):
    index=k.find(':')
    typ=k[:index]
    k=k[index+1:]    
    return k,typ



## Shape Element Function
#The function should take as input an iterparse Element object and return a dictionary.

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag=='node':
        for i in node_attr_fields:
            node_attribs[i]=element.attrib[i]
        
    if element.tag=='way':
        for i in way_attr_fields:
            way_attribs[i]=element.attrib[i]
    
    for tag in element.iter("tag"):
        dic={}
        attributes=tag.attrib
        if problem_chars.search(tag.attrib['k']):
            continue
        
        if element.tag=='node':
            dic['id']=node_attribs['id']
        else:
            dic['id']=way_attribs['id']
        
        dic['value'] = attributes['v']

        colon_k=LOWER_COLON.search(tag.attrib['k'])
        if colon_k:
            print colon_k.group(0)
            print tag.attrib['k']
            dic['key'],dic['type']=correct_k(tag.attrib['k'])
        else:
            dic['key']=attributes['k']
            dic['type']='regular'
        
        if attributes['k'] == 'addr:street':
            dic['value'] = update_name(attributes['v'],mapping)   
        else:
            dic['value'] = attributes['v']  
        
        if attributes['k'] == 'addr:postcode':
            dic['value'] = update_postcode(attributes['v']) 
        
        else:
            dic['value'] = attributes['v']
        
        #print dic
        tags.append(dic)

    if element.tag=='way':
        position=0
        for nd in element.iter("nd"):
            way_node_dic={}
            way_node_dic['id']=way_attribs['id']
            way_node_dic['node_id']=nd.attrib['ref']
            way_node_dic['position']=position
            position = position + 1
            way_nodes.append(way_node_dic)
         
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
from time import time
t0 = time()
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)
    print "Time to process: {} seconds".format(round(time()-t0, 3))






toll:automatic
toll:automatic
toll:automatic
toll:automatic
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
population:date
population:date
source:population
source:population
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
census:population
census:population
gnis:id
gnis:id
gnis:id
gnis:id
gnis